In [432]:
# Importation des bibliothèques Python
import pandas as pd
import os

In [433]:
# Charger les données
df = pd.read_csv('archive_raw/olympic_results.csv')

In [434]:
# Création de la liste des pays uniques, triés, et sauvegardée
unique_countries = pd.DataFrame(data=sorted(df['country_name'].unique()), columns=['country_name'])
unique_countries.to_csv('archive_raw/country_names.csv', index=False)

In [435]:
# Choix des slugs des jeux Olympiques (de 1948 à 2020)
game_slugs = [
    'tokyo-2020', 
    'rio-2016', 
    'london-2012',
    'beijing-2008',
    'athens-2004', 
    'sydney-2000', 
    'atlanta-1996', 
    'barcelona-1992', 
    'seoul-1988',
    'los-angeles-1984',
    'moscow-1980',
    'montreal-1976',
    'munich-1972',
    'mexico-city-1968',
    'tokyo-1964',
    'rome-1960',
    'melbourne-1956',
    'helsinki-1952',
    'london-1948'
]


In [436]:
# Filtrer les éditions d'été des JO avec les slugs
df = df[df['slug_game'].apply(lambda x: x in game_slugs)]

In [437]:
# Préparer la séparation des éditions pour les périodes 1948-1988 et 1992-2020 (JO 2020 ont eu lieu en 2021)
editions_before_1988 = [
    'seoul-1988', 'los-angeles-1984', 'moscow-1980', 'montreal-1976', 'munich-1972',
    'mexico-city-1968', 'tokyo-1964', 'rome-1960', 'melbourne-1956', 'helsinki-1952', 'london-1948'
]
editions_after_1988 = [
    'tokyo-2020', 'rio-2016', 'london-2012', 'beijing-2008', 'athens-2004', 'sydney-2000',
    'atlanta-1996', 'barcelona-1992'
]

In [438]:
# Préparer le filtrage des sports pour les périodes 1948-1988 et 1992-2020 (JO 2020 ont eu lieu en 2021)
sports_summer_before_1988 = [
    'Shooting', 'Diving', 'Canoe Sprint', 'Cycling Road', 'Football', 'Boxing', 'Basketball',
    'Cycling Track', 'Fencing', 'Water Polo', 'Wrestling', 'Artistic Gymnastics', 'Weightlifting',
    'Modern Pentathlon', 'Hockey', 'Athletics', 'Swimming', 'Sailing', 'Rowing'
]
sports_summer_after_1988 = [
    'Shooting', 'Diving', 'Canoe Sprint', 'Cycling Road', 'Football', 'Boxing', 'Basketball',
    'Cycling Track', 'Fencing', 'Table Tennis', 'Badminton', 'Water Polo', 'Wrestling',
    'Artistic Gymnastics', 'Canoe Slalom', 'Rhythmic Gymnastics', 'Weightlifting', 'Modern Pentathlon',
    'Hockey', 'Volleyball', 'Artistic Swimming', 'Athletics', 'Swimming', 'Sailing', 'Rowing',
    'Tennis', 'Equestrian', 'Archery', 'Handball', 'Judo'
]

In [439]:
# Filtrer les sports pour les périodes 1948-1988 et 1992-2020 (JO 2020 ont eu lieu en 2021)
df_summer_before_1988 = df[df['discipline_title'].apply(lambda x: x in sports_summer_before_1988)]
df_summer_before_1988 = df_summer_before_1988[df_summer_before_1988['slug_game'].apply(lambda x: x in editions_before_1988)]

df_summer_after_1988  = df[df['discipline_title'].apply(lambda x: x in sports_summer_after_1988)]
df_summer_after_1988  = df_summer_after_1988[df_summer_after_1988['slug_game'].apply(lambda x: x in editions_after_1988)]

In [440]:
# Extraire les rangs pour les positions <= n (par défaut : 10)
def extract_n_first(df, n_athletes=10):
    df['rank_position'] = pd.to_numeric(df['rank_position'], errors='coerce')
    df_cl = df.dropna(subset=['rank_position'])
    df_cl['rank_position'] = df_cl['rank_position'].astype(int)
    df_cl = df_cl[df_cl['rank_position'] <= n_athletes]
    return df_cl

# Application de la fonction avec n_athletes, groupement par pays et par édition
n_athletes = 10

filtered_df_before = extract_n_first(df_summer_before_1988, n_athletes)
df_before          = filtered_df_before.groupby(by=['country_name', 'slug_game']).count().reset_index()

filtered_df        = extract_n_first(df_summer_after_1988, n_athletes)
df_after           = filtered_df.groupby(by=['country_name', 'slug_game']).count().reset_index()

df_clean_before    = df_before[['country_name', 'rank_position', 'slug_game']].copy()
df_clean_after     = df_after [['country_name', 'rank_position', 'slug_game']].copy()

C:\Users\Julien\AppData\Local\Temp\ipykernel_25480\645130061.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cl['rank_position'] = df_cl['rank_position'].astype(int)
C:\Users\Julien\AppData\Local\Temp\ipykernel_25480\645130061.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cl['rank_position'] = df_cl['rank_position'].astype(int)


In [441]:
# Extraire l'année et créer un tableau croisé dynamique en une seule étape
df_clean_before['year'] = df_clean_before['slug_game'].str.extract(r'(\d{4})')
df_pivot = df_clean_before.pivot_table(index='country_name', columns='year', values='rank_position', aggfunc='first')

df_pivot.reset_index(inplace=True)

# Remplacer les valeurs manquantes par 0
df_pivot.fillna(0, inplace=True)

# Transposer et réinitialiser l'index, puis utiliser la première ligne comme noms de colonnes
df_pivot         = df_pivot.transpose()
df_pivot         = df_pivot.iloc[0:].reset_index(drop=True)
df_pivot.columns = df_pivot.iloc[0]
first_part       = df_pivot.drop(0).reset_index(drop=True)

In [442]:
# Extraire l'année et créer un tableau croisé dynamique en une seule étape
df_clean_after['year'] = df_clean_after['slug_game'].str.extract(r'(\d{4})')
df_pivot = df_clean_after.pivot_table(index='country_name', columns='year', values='rank_position', aggfunc='first')

df_pivot.reset_index(inplace=True)

# Remplacer les valeurs manquantes par 0
df_pivot.fillna(0, inplace=True)

# Transposer et réinitialiser l'index, puis utiliser la première ligne comme noms de colonnes
df_pivot         = df_pivot.transpose()
df_pivot         = df_pivot.iloc[0:].reset_index(drop=True)
df_pivot.columns = df_pivot.iloc[0]
second_part      = df_pivot.drop(0).reset_index(drop=True)

In [443]:
# Définition des années pour les fichiers finaux
years = [1948, 1952, 1956, 1960, 1964, 1968, 1972, 1976, 1980, 1984, 
         1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020]

In [444]:
# Ajout des années et organisation de first_part pour 1948-1988
first_part.rename(columns={'Unnamed: 0': 'year'}, inplace=True)
first_part['year'] = years[:11]
first_part.set_index('year', inplace=True)

# Ajout des années et organisation de second_part pour 1992-2020 (JO 2020 ont eu lieu en 2021)
second_part.rename(columns={'Unnamed: 0': 'year'}, inplace=True)
second_part['year'] = years[11:]
second_part.set_index('year', inplace=True)

first_part.to_csv('first_part.csv')
second_part.to_csv('second_part.csv')


In [445]:
# Sauvegarde de la version brute de la liste des pays
country_list = second_part.columns[:].tolist()
country_list.sort()
with open('country_list_brut.txt', 'w') as f:
    for country in country_list:
        f.write(f"{country}\n")

In [ ]:
# Mapping des noms de pays et nettoyage
name_mapping = {
    "Côte d'Ivoire" : "Cote d'Ivoire",
    "Democratic People's Republic of Korea" : "North Korea",
    "Hong Kong, China" : "Hong Kong China",
    "Islamic Republic of Iran" : "Iran",
    "People's Republic of China" : "China",
    "Republic of Korea" : "South Korea",
    "Republic of Moldova" : "Moldova",
    "Russian Federation" : "Russia",
    "Saint Kitts and Nevis" : "St. Kitts and Nevis",
    "Saint Lucia" : "St. Lucia",
    "Swaziland" : "Eswatini",
    "Syrian Arab Republic" : "Syria",
    "United Arab Emirates" : "UAE",
    "United Republic of Tanzania" : "Tanzania",
    "United States of America" : "USA",
    "Congo" : "Congo Rep.",
    "Democratic Republic of the Congo" : "Congo Dem. Rep."
}

# Liste des pays à supprimer dans la liste des délégations olympiques:
to_del = [
    "Virgin Islands, British", 
    "US Virgin Islands", 
    "The Former Yugoslav Republic of Macedonia",
    "Serbia and Montenegro", 
    "Refugee Olympic Team", 
    "Refugee Olympic Athletes", 
    "ROC", 
    "Puerto Rico", 
    "Netherlands Antilles", 
    "Kosovo",
    "Independent Olympic Athletes", 
    "Guam", 
    "Czechoslovakia", 
    "Chinese Taipei", 
    "Bermuda", 
    "Aruba", 
    "American Samoa"
]

'\n+Afghanistan\n+Antigua and Barbuda\n+Belize\n+Bhutan\n+Bolivia\n+Brunei\n+Cambodia\n+Chad\n+Comoros\n+Dominica\n+Equatorial Guinea\n+Gambia\n+Guinea\n+Lao\n+Lesotho\n+Libya\n+Malawi\n+Maldives\n+Marshall Islands\n+Mauritania\n+Nepal\n+Niger\n+Palau\n+Palestine\n+Sao Tome and Principe\n+South Sudan\n+St. Vincent and the Grenadines\n+Timor-Leste\n+Tuvalu\n+Vanuatu\n+Yemen\n'

In [447]:
# Copie et nettoyage des données de second_part pour les noms de pays et les colonnes à supprimer
second_part_ = second_part.copy()
second_part_ = second_part_.rename(columns=name_mapping)
second_part_ = second_part_.drop(columns=[col for col in second_part_.columns if col in to_del])

In [448]:
# Sauvegarde de la version nettoyée de la liste des pays
country_list = second_part_.columns[:].tolist()
country_list.sort()
with open('list_post_traitement.txt', 'w') as f:
    for country in country_list:
        f.write(f"{country}\n")

In [449]:
# Sauvegarde de la version nettoyée de second_part 
second_part_.to_csv('second_part_countries_cleaned.csv')